In [1]:
import warnings
warnings.filterwarnings("ignore")
import math
import pymysql
import datetime
import tensorflow as tf

In [2]:
class VolumeData(object):
    def __init__(self):
        conn = pymysql.connect(host = '219.224.169.45', 
                               user = 'lizimeng', 
                               password = 'codegeass', 
                               db='commodity')
        cursor = conn.cursor()
        # 上海2014年开始的交易量数据
        sql = "select code,date,sum(volume) from contract_daily where date>='20140101' and \
        code in (select code from code2exchange where exchange='SHFE') group by code,date order by date asc"
        self.volume = {}
        self.days = []
        self.codes = []
        cursor.execute(sql)
        for row in cursor.fetchall():
            c,d,v = row
            if c not in self.codes:
                self.codes.append(c)
            if d not in self.codes:
                self.days.append(d)
            assert (c,d) not in self.volume
            self.volume[(c,d)] = v
        cursor.close()
        conn.close()
        
    def get_data1(self, code, day, val):
        '''
        required: code is in self.codes and day+i in self.days for i in [0,1,...,val-1]
        modified: none
        function: 品种code从day起val个交易日（包括day）的结算价序列
        '''
        bx = self.days.index(day)
        result = []
        for i in range(val):
            result.append(self.volume[(code,self.days[bx+i])])
        return result
    
    def get_data2(self, day):
        '''
        required: day in self.days
        modified: none
        function: 全部品种在交易日day的结算价序列
        '''
        result = []
        for code in self.codes:
            result.append(self.volume[(code,day)])
        return result
        
    def get_data3(self, code, day):
        '''
        function: 品种code在交易日day的结算价（一个数字）
        '''
        assert code in self.codes
        assert day in self.days
        return self.volume[(code,day)]

In [3]:
class Prediction(object):
    vd = VolumeData()
    @classmethod
    def linearRegression(cls, day, pwind=5, twind=150):
        vd = cls.vd
        days = cls.vd.days
        codes = cls.vd.codes
        # 模型构建
        x = tf.placeholder(dtype=tf.float64, shape=[None, len(codes)*pwind+1])
        w = tf.Variable(tf.zeros(shape=[len(codes)*pwind+1, 1], dtype=tf.float64))
        y = tf.matmul(x,w)
        y_ = tf.placeholder(dtype=tf.float64, shape=[None,1])
        loss = tf.log(tf.reduce_sum((y-y_)**2))
        train = tf.train.GradientDescentOptimizer(0.00003).minimize(loss)
        # 生成训练样例
        Tx = []
        Ty = []
        for i in range(days.index(day)-twind, days.index(day)-pwind):
            tmp = []
            d = days[i]
            for c in codes:
                tmp += vd.get_data1(c, d, pwind)
            tmp.append(1.)
            Tx.append(tmp)
            Ty.append([vd.get_data3('cu', days[i+pwind])])
        # 确定预测的输入
        Px = []
        d = days[days.index(day) - pwind]
        for c in codes:
            Px += vd.get_data1(c, d, pwind)
        Px.append(1.)
        Px = [Px]
        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)
            for i in range(2*10**5):
                sess.run(train, feed_dict={x:Tx, y_:Ty})
            res = sess.run(y, feed_dict={x:Px})
            print (sess.run(loss, feed_dict={x:Tx, y_:Ty}))
        return res[0][0]
    
    @classmethod
    def MLP1(cls, day, pwind=5, twind=150):
        '''
        双隐层的前馈神经网络
        输入到第一个隐层为部分链接，同品种不同交易日链接到同一个神经元
        隐层之间为全连接
        激活函数为sigmoid
        '''
        vd = cls.vd
        days = cls.vd.days
        codes = cls.vd.codes
        # 模型构建
        x = tf.placeholder(dtype=tf.float64, shape=[None, len(codes)*pwind+1])
        w = tf.Variable(tf.zeros(shape=[len(codes)*pwind+1, 1], dtype=tf.float64))
        y = tf.matmul(x,w)
        y_ = tf.placeholder(dtype=tf.float64, shape=[None,1])
        loss = tf.log(tf.reduce_sum((y-y_)**2))
        train = tf.train.GradientDescentOptimizer(0.00003).minimize(loss)
        pass
        
    @classmethod
    def MLP2(cls, day, pwind=5, twind=150):
        '''
        双隐层的前馈神经网络
        输入到第一个隐层为部分链接，同交易日不同品种链接到同一个神经元
        隐层之间为全连接
        激活函数为sigmoid
        '''
        pass


In [4]:
p = Prediction
d = datetime.date(2017,7,10)
print (p.linearRegression(d))
print (p.vd.get_data3('cu', d))

28.9087342041
375420.509719
294256
